In [1]:
import requests
key = 'RGAPI-b9ac04aa-42e0-4f43-8e98-bd8f78cac878'

## Requesting data from RIOT

In [2]:
player_data_url = 'https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/ur%20mom%20sus'
player_data_url = player_data_url +'?api_key=' + key
r = requests.get(player_data_url)
my_info = r.json()
print(my_info)

{'id': 'TGpB7SIsSLFgjRW7IqKjaDOnXJjZLRtOxggozO3-JJtD4YO4', 'accountId': '6t3jE1Tjo9QTawW-wfDbJN3qPawFVUZDYsWE-F84dfAeE4sfYMcZY-ky', 'puuid': 'mPnEBBd82jEPIxTYp4IRLyZrvX4mY7ZqJOcXZRHRascKaGQYwSoYGIOkL3r_OroVZICmnnb_WZ10Ow', 'name': 'ur mom sus', 'profileIconId': 907, 'revisionDate': 1697774443000, 'summonerLevel': 118}


## Getting puuid of my account

In [3]:
my_puuid = my_info['puuid']
my_puuid

'mPnEBBd82jEPIxTYp4IRLyZrvX4mY7ZqJOcXZRHRascKaGQYwSoYGIOkL3r_OroVZICmnnb_WZ10Ow'

In [4]:
match_data_url = 'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/mPnEBBd82jEPIxTYp4IRLyZrvX4mY7ZqJOcXZRHRascKaGQYwSoYGIOkL3r_OroVZICmnnb_WZ10Ow/ids?start=0&count=20'
# making request...
match_data_url = match_data_url +'&api_key=' + key
r = requests.get(match_data_url)
matches = r.json()
# we will be working with the first (latest) match
match = matches[0]

In [5]:
match

'NA1_4805495584'

## After getting the match ID, we will now be making another request to get the data of that specific match.

In [6]:
match_url = 'https://americas.api.riotgames.com/lol/match/v5/matches/'
match_url = match_url + match + '?api_key=' + key
r = requests.get(match_url)
match_data = r.json()
# Taking a look at our keys in the dict
match_data.keys()

dict_keys(['metadata', 'info'])

In [7]:
print(match_data['info'].keys())
len(match_data['info']['participants'])

dict_keys(['gameCreation', 'gameDuration', 'gameEndTimestamp', 'gameId', 'gameMode', 'gameName', 'gameStartTimestamp', 'gameType', 'gameVersion', 'mapId', 'participants', 'platformId', 'queueId', 'teams', 'tournamentCode'])


10

### Now we will use our puuid, to find where my ID is located, and then getting my satistics with a direct call to index position

In [8]:
my_index = match_data['metadata']['participants'].index(my_puuid)
# using my index position, to see what champion I played in this match.
match_data['info']['participants'][my_index]['championName']

'Tristana'

# Below we will be making functions to automate the above:

In [12]:
# The first function simply gets the puuid, given a summoner name and region
# This is exactly the same as our first example, except we're building the API URL from scratch
def get_puuid(summoner_name, region, api_key):
    api_url = (
        "https://" + 
        region +
        ".api.riotgames.com/lol/summoner/v4/summoners/by-name/" +
        summoner_name +
        "?api_key=" +
        api_key
    )

    print(api_url)
    
    resp = requests.get(api_url)
    player_info = resp.json()
    puuid = player_info['puuid']
    return puuid 

In [13]:
summoner_name = 'ur%20mom%20sus'
region = 'na1'

puuid = get_puuid(summoner_name, region, key)
puuid

https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/ur%20mom%20sus?api_key=RGAPI-b9ac04aa-42e0-4f43-8e98-bd8f78cac878


'mPnEBBd82jEPIxTYp4IRLyZrvX4mY7ZqJOcXZRHRascKaGQYwSoYGIOkL3r_OroVZICmnnb_WZ10Ow'

In [14]:
# The function to get a list of all the match IDs given a players puuid and mass region
def get_match_ids(puuid, mass_region, api_key):
    api_url = (
        "https://" +
        mass_region +
        ".api.riotgames.com/lol/match/v5/matches/by-puuid/" +
        puuid + 
        "/ids?start=0&count=20" + 
        "&api_key=" + 
        api_key
    )
    
    print(api_url)
    
    resp = requests.get(api_url)
    match_ids = resp.json()
    return match_ids  

In [16]:
# NOTE: region and mass_region are different
# for instance, NA1 is the North American region
# which is part of the AMERICAS mass region
# EUW1 is Europe West region, part of the EUROPE mass region
mass_region = 'AMERICAS'

match_ids = get_match_ids(puuid, mass_region, key)
match_ids

https://AMERICAS.api.riotgames.com/lol/match/v5/matches/by-puuid/mPnEBBd82jEPIxTYp4IRLyZrvX4mY7ZqJOcXZRHRascKaGQYwSoYGIOkL3r_OroVZICmnnb_WZ10Ow/ids?start=0&count=20&api_key=RGAPI-b9ac04aa-42e0-4f43-8e98-bd8f78cac878


['NA1_4805495584',
 'NA1_4805481033',
 'NA1_4805457884',
 'NA1_4805431895',
 'NA1_4801022254',
 'NA1_4801008090',
 'NA1_4800965281',
 'NA1_4798822603',
 'NA1_4798756803',
 'NA1_4798712876',
 'NA1_4798654486',
 'NA1_4795277925',
 'NA1_4794289367',
 'NA1_4794266519',
 'NA1_4794248629',
 'NA1_4794226994',
 'NA1_4794188661',
 'NA1_4793335450',
 'NA1_4793302365',
 'NA1_4793273642']

In [17]:
# From a given match ID and mass region, get the data about the game
def get_match_data(match_id, mass_region, api_key):
    api_url = (
        "https://" + 
        mass_region + 
        ".api.riotgames.com/lol/match/v5/matches/" +
        match_id + 
        "?api_key=" + 
        api_key
    )
    
    resp = requests.get(api_url)
    match_data = resp.json()
    return match_data  

In [19]:
match_id = match_ids[0]
match_data = get_match_data(match_id, mass_region, key)
match_data

{'metadata': {'dataVersion': '2',
  'matchId': 'NA1_4805495584',
  'participants': ['HJlUxY_HI-qUMCyiR7CLY-8F0tqR2AzsNHb0PIQFsONeSmxVoZ6Zl2bl_YrhnndHQqfOuT6S8Me3kQ',
   'Nqre2jqgXoz4ZTdLtdcYhmx0eq6yiIpF5L1qKYb7pLABZ9HD1Owy7aswHYmOnIqpm_mm1UWNSEXwtA',
   'OOwk3KLX-xVNhm0hB3NV7LSufTKcH54dxNWhzceopHwA7ZImsSfhozFD3lwyt6ePLdcveDQz7Flvig',
   'crC0meQX96kBGv5ckeqP6HjNm7OyXysAMiGEyLpbfF2PAvinRX1IbY5AwKECQlDerMnS7q3DBGKiHA',
   'XTH75rKbd-ntz07UyVr5M7Zt1bz2COMo7QLgVkFrE8KTOCkc3ibTXpnoSDWhfNDyP2-5uVJkDllULQ',
   '3mZMBVTYVBPnEKI41IAA7uPW1c5J_5w-vti4kKhCltGwf3dXIprJ64d-oBpxeaLaYKxqVwwDKHjsvQ',
   'HYs_cnziuhXB-YgnmLM8yy9WacEVNPHISbX6FklzlkozwfyVoErm8STofB18Jwd9XrKa4rD02dZKJA',
   'mPnEBBd82jEPIxTYp4IRLyZrvX4mY7ZqJOcXZRHRascKaGQYwSoYGIOkL3r_OroVZICmnnb_WZ10Ow',
   'Y8mERO0zb6mb5j-MmHImzR6v3cr6aOIF46i9Hkn-fALTZxmiJeRZkI3Dc4yWhYekfxjGM4OOU77o6w',
   'xViEjVJVLO1fHdTuj7VKaGA4vLzlzT02s-gWJPi6gAxAa8k3q0H7EIwLfxZR_nTsBldBkNyPUJHfkA']},
 'info': {'gameCreation': 1697772816101,
  'gameDuration': 1604,
  

In [20]:
# Given the match data and a players puuid, return the data about just them
def find_player_data(match_data, puuid):
    participants = match_data['metadata']['participants']
    player_index = participants.index(puuid)
    player_data = match_data['info']['participants'][player_index]
    return player_data

In [21]:
find_player_data(match_data, puuid)

{'allInPings': 0,
 'assistMePings': 0,
 'assists': 24,
 'baitPings': 0,
 'baronKills': 0,
 'basicPings': 0,
 'bountyLevel': 0,
 'challenges': {'12AssistStreakCount': 0,
  'abilityUses': 124,
  'acesBefore15Minutes': 2,
  'alliedJungleMonsterKills': 0,
  'baronTakedowns': 0,
  'blastConeOppositeOpponentCount': 0,
  'bountyGold': 2025,
  'buffsStolen': 0,
  'completeSupportQuestInTime': 0,
  'controlWardsPlaced': 0,
  'damagePerMinute': 2474.3475781912007,
  'damageTakenOnTeamPercentage': 0.18231080765409652,
  'dancedWithRiftHerald': 0,
  'deathsByEnemyChamps': 16,
  'dodgeSkillShotsSmallWindow': 1,
  'doubleAces': 0,
  'dragonTakedowns': 0,
  'earlyLaningPhaseGoldExpAdvantage': 0,
  'effectiveHealAndShielding': 0,
  'elderDragonKillsWithOpposingSoul': 0,
  'elderDragonMultikills': 0,
  'enemyChampionImmobilizations': 12,
  'enemyJungleMonsterKills': 0,
  'epicMonsterKillsNearEnemyJungler': 0,
  'epicMonsterKillsWithin30SecondsOfSpawn': 0,
  'epicMonsterSteals': 0,
  'epicMonsterStolenW

In [33]:
# We initialise an empty dictionary to store data for each game
data = {
    'champion': [],
    'kills': [],
    'deaths': [],
    'assists': [],
    'win': []
}

for match_id in match_ids:
    print('Extracting data for: ' +match_id)
    
    # run the two functions to get the player data from the match ID
    match_data = get_match_data(match_id, mass_region, key)
    player_data = find_player_data(match_data, puuid)
    
    # assign the variables we're interested in
    champion = player_data['championName']
    k = player_data['kills']
    d = player_data['deaths']
    a = player_data['assists']
    win = player_data['win']
     
    # add them to our dataset
    data['champion'].append(champion)
    data['kills'].append(k)
    data['deaths'].append(d)
    data['assists'].append(a)
    data['win'].append(win) 

Extracting data for: NA1_4805495584
Extracting data for: NA1_4805481033
Extracting data for: NA1_4805457884
Extracting data for: NA1_4805431895
Extracting data for: NA1_4801022254
Extracting data for: NA1_4801008090
Extracting data for: NA1_4800965281
Extracting data for: NA1_4798822603
Extracting data for: NA1_4798756803
Extracting data for: NA1_4798712876
Extracting data for: NA1_4798654486
Extracting data for: NA1_4795277925
Extracting data for: NA1_4794289367
Extracting data for: NA1_4794266519
Extracting data for: NA1_4794248629
Extracting data for: NA1_4794226994
Extracting data for: NA1_4794188661
Extracting data for: NA1_4793335450
Extracting data for: NA1_4793302365
Extracting data for: NA1_4793273642


In [24]:
# Data on the last 20 games of the account in question
data

{'champion': ['Tristana',
  'Ezreal',
  'Sett',
  'Jhin',
  'Varus',
  'Pyke',
  'Thresh',
  'Skarner',
  'Amumu',
  'Trundle',
  'Poppy',
  'Ryze',
  'Viego',
  'Gragas',
  'Varus',
  'Karthus',
  'Annie',
  'Pyke',
  'Thresh',
  'Lux'],
 'kills': [20, 5, 6, 19, 47, 4, 1, 5, 10, 2, 2, 6, 13, 6, 12, 13, 7, 9, 1, 16],
 'deaths': [16,
  6,
  15,
  6,
  16,
  2,
  12,
  3,
  8,
  7,
  12,
  6,
  10,
  7,
  5,
  15,
  17,
  9,
  7,
  11],
 'assists': [24,
  12,
  7,
  26,
  42,
  8,
  21,
  12,
  30,
  11,
  16,
  7,
  14,
  24,
  30,
  25,
  31,
  20,
  11,
  22],
 'win': [False,
  False,
  False,
  True,
  True,
  True,
  False,
  True,
  False,
  False,
  False,
  False,
  False,
  True,
  True,
  False,
  True,
  False,
  False,
  True]}

In [30]:
# Ultimate function to get all data:
def gather_all_data(puuid, match_ids, mass_region, api_key):
    # We initialise an empty dictionary to store data for each game
    data = {
        'champion': [],
        'kills': [],
        'deaths': [],
        'assists': [],
        'win': []
    }

    for match_id in match_ids:
        print(match_id)

        # run the two functions to get the player data from the match ID
        match_data = get_match_data(match_id, mass_region, api_key)
        player_data = find_player_data(match_data, puuid)

        # assign the variables we're interested in
        champion = player_data['championName']
        k = player_data['kills']
        d = player_data['deaths']
        a = player_data['assists']
        win = player_data['win']

        # add them to our dataset
        data['champion'].append(champion)
        data['kills'].append(k)
        data['deaths'].append(d)
        data['assists'].append(a)
        data['win'].append(win)    
    
    df = pd.DataFrame(data)
    
    return df

# Now we will use pandas to import it into a DataFrame

In [26]:
import pandas as pd

In [28]:
df = pd.DataFrame(data)
df

,champion,kills,deaths,assists,win
0,Tristana,20,16,24,False
1,Ezreal,5,6,12,False
2,Sett,6,15,7,False
3,Jhin,19,6,26,True
4,Varus,47,16,42,True
5,Pyke,4,2,8,True
6,Thresh,1,12,21,False
7,Skarner,5,3,12,True
8,Amumu,10,8,30,False
9,Trundle,2,7,11,False


In [32]:
df = gather_all_data(puuid, match_ids, mass_region, key)

NA1_4805495584
NA1_4805481033
NA1_4805457884
NA1_4805431895
NA1_4801022254
NA1_4801008090
NA1_4800965281
NA1_4798822603
NA1_4798756803
NA1_4798712876
NA1_4798654486
NA1_4795277925
NA1_4794289367
NA1_4794266519
NA1_4794248629
NA1_4794226994
NA1_4794188661
NA1_4793335450
NA1_4793302365
NA1_4793273642


In [35]:
# converting boolean win column into 0/1 
df['win'] = df['win'].astype(int)
df

,champion,kills,deaths,assists,win
0,Tristana,20,16,24,0
1,Ezreal,5,6,12,0
2,Sett,6,15,7,0
3,Jhin,19,6,26,1
4,Varus,47,16,42,1
5,Pyke,4,2,8,1
6,Thresh,1,12,21,0
7,Skarner,5,3,12,1
8,Amumu,10,8,30,0
9,Trundle,2,7,11,0
